In [ ]:
import sys, os

In [ ]:
import getpass

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

user = getpass.getuser()

if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"

In [ ]:
#import data!
from sample.DataIO import DataIO 

data = DataIO(sc,feature_path=parquet_path+"/featureDataCvr.parquet",company_path=parquet_path+"/companyCvrData")
feature_data = data.mergeCompanyFeatureData()
small_data = feature_data.limit(50).cache()

In [ ]:
#Select parameters
from sample.CreateParameters import CreateParameters


params = CreateParameters(small_data.columns)

In [ ]:
params.select_params()

In [ ]:
#print(params.export_values())
dicKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}
dicBKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': False, 'clusters': 2, 'tolerance': 0.0001}
dicGMM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': True, 'clusters': 10, 'tolerance': 0.0001}
#{'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'initialstep': 10, 'standardize': True, 'clusters': 3, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'Prediction', 'iterations': 10}

In [ ]:
from sample.ExecuteWorkflow import ExecuteWorkflow

execution_model = ExecuteWorkflow()
#print(execution_model.params)
execution_model.params = dicGMM#params.export_values()

In [ ]:
from pyspark.sql import functions as F

runTest = execution_model.run(data=small_data)

typeUdf = F.udf(lambda x: type(x))
runTest.select(typeUdf(F.col("scaled_features"))).distinct().show(truncate = False)

In [ ]:
from sample.ShowKmeans import ShowResults

results = ShowResults(execution_model.params)
results.select_prototypes(runTest)

In [ ]:
import pyspark.ml.feature as features

In [ ]:
scaling_model = features.StandardScaler(
                inputCol='features',
                outputCol="scaled_features",
                withMean=True,
                withStd=True
            )

In [ ]:
from pyspark.ml.linalg import Vectors
data = [(Vectors.sparse(3, {0:4.0}),),
        (Vectors.sparse(3, {1:4.0}),),
        (Vectors.dense([0.9, 0.8, -0.05]),),
        (Vectors.dense([0.75, 0.935, -0.05]),),
        (Vectors.dense([-0.83, -0.68, -0.05]),),
        (Vectors.dense([-0.91, -0.76, -0.05]),)]

df = spark.createDataFrame(data, ["features"])

In [ ]:
test = scaling_model.fit(df)

In [ ]:
test.transform(df).show()